Het uitvoeren van een machine learning model met extra features 

Het doel van het notebook is om te kijken of het machine learning model een hogere r2 en een lagere root mean squared error heeft als de feature globale straling en percentage zonuren is toegevoegd.

Uit verschillende onderzoeken blijkt dat naast het aantal zonuren de zonnestraling een belangrijke feature is (Gupta, 2022), (Zhao, 2012). De KNMI heeft ook data beschikbaar over de globale straling en deze is gebruikt in het model. Ook heeft de KNMI data beschikbaar over het percentage zonuren op een dag, deze feature is ook meegenomen in het model. 

In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

De dataset waarbij de waarde van de laatste maand heeft wordt opgehaald en leesbaar gemaakt. Alle features worden in het model meegenomen.

In [54]:
dataset = pd.read_csv('Data_laatste_maand.csv')

In [55]:
dataset.drop(columns=['Unnamed: 0'], inplace=True)
dataset

,Datum,NettoVerbruikBerekend_30,Temp_last_month,Wind_last_month,Zon_last_month,Infl_last_month,Q_last_month,SP_last_month
0,2000-01,8847,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-02,8235,4.329032,3.980645,1.806452,52.22,201.774194,21.838710
2,2000-03,8738,5.855172,4.296552,3.682759,52.29,484.310345,36.827586
3,2000-04,7912,6.796774,3.677419,2.583871,52.73,629.258065,21.967742
4,2000-05,8221,9.986667,2.840000,5.116667,52.74,1263.600000,37.066667
...,...,...,...,...,...,...,...,...
259,2021-08,8981,17.987097,2.416129,6.264516,115.79,1674.290323,38.677419
260,2021-09,9030,16.935484,2.677419,5.325806,117.17,1391.612903,36.258065
261,2021-10,9410,15.930000,2.160000,5.940000,123.29,1165.900000,46.433333
262,2021-11,9678,11.612903,3.045161,3.945161,136.56,619.709677,36.774194


Doordat de waarde een plaats is opgeschoven is er voor de features in januari 200 geen waarde bekend. Met de dropna functie wordt deze rij uit de dataset gehaald.

In [56]:
dataset.dropna(inplace=True)

Met de correlatie functie wordt er gekeken of er een relatie is tussen de features. Wanneer de waarde negatief is, is er sprake van een negatieve relatie. De relatie is sterk wanneer deze boven de 0.5 of onder de -0.5 ligt. Hoe dichter de waarde bij de 1 of -1 ligt, hoe sterker de relatie is. Uit de resultaten van het correlatie model blijkt dat de zonnestraling (Q_last_month) een sterke relatie heeft met het energie verbruik en zelfs een sterkere relatie heeft dan het aantal zonuren. Het percentage zonuren (SP_last_month) heeft een wat minder sterke relatie.

In [57]:
dataset.corr()

,NettoVerbruikBerekend_30,Temp_last_month,Wind_last_month,Zon_last_month,Infl_last_month,Q_last_month,SP_last_month
NettoVerbruikBerekend_30,1.000000,-0.421654,0.212029,-0.603668,0.382205,-0.660923,-0.471734
Temp_last_month,-0.421654,1.000000,-0.598220,0.740483,0.021698,0.816944,0.542551
Wind_last_month,0.212029,-0.598220,1.000000,-0.543774,-0.061670,-0.569440,-0.448990
Zon_last_month,-0.603668,0.740483,-0.543774,1.000000,0.049243,0.947282,0.921413
Infl_last_month,0.382205,0.021698,-0.061670,0.049243,1.000000,0.017762,0.072853
Q_last_month,-0.660923,0.816944,-0.569440,0.947282,0.017762,1.000000,0.754597
SP_last_month,-0.471734,0.542551,-0.448990,0.921413,0.072853,0.754597,1.000000


De eerste stap in een regressie model is het bepalen van de X en y waarden. Vervolgens wordt de data verdeeld in een test dataset en een train dataset. Door dit te doen kan er met die waarden gecontroleerd worden of het model goed voorspeld. Met de r2 score wordt berekent hoeveel variantie er in de features wordt verklaard door de energieconsumptie. Hoe dichter de waarde bij de 1 ligt, hoe meer de features verklaard worden door de energieconsumptie. De root mean squared error geeft het verschil weer tussen de voorspelde en werkelijke waarde weer. 

In [58]:
X = dataset[['Temp_last_month', 'Wind_last_month', 'Zon_last_month', 'Infl_last_month', 'Q_last_month', 'SP_last_month']].values
y = dataset['NettoVerbruikBerekend_30'].values 

In [59]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [60]:
LR = LinearRegression()
LR.fit(x_train,y_train)
y_prediction =  LR.predict(x_test)
y_prediction

array([ 8844.02459693,  9518.56989871,  9314.02674051,  9950.47458658,
        8744.91747826,  9908.0689009 ,  9537.42330053,  9900.907135  ,
        9147.50595958,  8640.77115572,  9135.57773692,  9094.94325435,
        9549.53702469,  9289.7370485 ,  9452.93206397, 10111.0892182 ,
        9047.02899663,  8264.93284485,  8762.50780748,  8901.37644747,
        9617.66797307,  8850.87088951,  8569.08982233,  9290.80428742,
        9993.83617889,  8555.26894418,  8957.27568339,  9420.15263308,
        8708.20579597,  8305.90365307,  8307.41485035,  8825.872588  ,
        8928.71369177,  9238.14291013,  9183.00194857,  9723.17472999,
        9658.18258894, 10022.77631411,  8712.61167629,  9142.08642702,
        9640.9623777 ,  9177.65073727,  9577.30350972,  8571.96364705,
        8749.29288783,  9487.78354846,  9735.84788777,  8305.76858289,
        9223.61014943,  9792.641588  ,  9435.86065859,  9118.56560255,
        8747.65014542])

In [61]:
score=r2_score(y_test,y_prediction)
print('r2 score = ',score)
print('root_mean_squared error of = ',np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score =  0.6911087303012194
root_mean_squared error of =  337.7489760992334


In de tabel hieronder zijn de resultaten te zien van de r2 en de root mean squered error wanneer er met de features gevarieerd wordt. In de X zijn de features temperatuur, wind en inflatie altijd meegenomen, alleen de feature zonuren, zonnestraling (Q) en percentage zonuren (SP) variëren. Uit de tabel kan geconcludeerd worden dat het machine learning model het best presteert wanneer alle zon features (zonuren, zonnestraling (Q) en percentage zonuren (SP)) zijn meegenomen. Wat daarnaast opvalt is dat de feature percentage zonuren niet veel toevoegt en op zichzelf ook niet goed presteert.

|  | r2 | Root mean squared error |
| --- | --- | --- |
| Zonuren | 0.48 | 437.12 |
| Zonuren, Q | 0.66 | 354.17 |
| Zonuren, SP | 0.58 | 392.83 |
| Zonuren, Q, SP | 0.69 | 337.75 |
| Q | 0.65 | 359.80 |
| SP | 0.37 | 481.15 |
| Q & SP | 0.65 | 357.74 |

Bronnenlijst
Gupta, A. (2022, augustus 23). Python | Linear Regression using sklearn. Opgehaald van geeks for geeks: https://www.geeksforgeeks.org/python-linear-regression-using-sklearn/
Zhao, H.-x. (2012, april 27). A review on the prediction of building energy consumption. Opgehaald van ScienceDirect: https://www.sciencedirect.com/science/article/pii/S1364032112001438